# Introduction à PyTorch – Classification avec un Réseau de Neurones

**Objectif** : Apprendre à construire, entraîner et évaluer un réseau de neurones avec **PyTorch** sur le dataset **Iris**.

---

## Compétences acquises

| Étape | Ce que vous saurez faire |
|-------|---------------------------|
| 1 | Charger et préparer des données |
| 2 | Créer un `Dataset` et `DataLoader` |
| 3 | Définir un modèle avec plusieurs couches |
| 4 | Entraîner avec une boucle propre |
| 5 | Évaluer et visualiser les performances |

---

**Exécutez chaque cellule dans l’ordre.**

## 0. Imports

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Configuration du device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device utilisé : {DEVICE}")

## 1. Hyperparamètres (à modifier plus tard)

In [ ]:
# MODIFIEZ CES VALEURS POUR EXPÉRIMENTER !
HIDDEN_SIZE = 50       # Taille des couches cachées
NUM_LAYERS = 3         # Nombre de couches cachées
DROPOUT_PROB = 0.2     # Probabilité de dropout
LEARNING_RATE = 0.01   # Taux d'apprentissage
NUM_EPOCHS = 50        # Nombre d'epochs d'entraînement
BATCH_SIZE = 50        # Taille des mini-batchs

## 2. Chargement et préparation des données (Iris)

In [ ]:
# Chargement du dataset Iris
X, y = load_iris(return_X_y=True)
X = X.astype(np.float32)
y = y.astype(np.int64)

# Split : 90% train, 10% validation
X_train, X_dev, y_train, y_dev = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y
)

print(f"Train : {X_train.shape}, Dev : {X_dev.shape}")

## 3. Création d’un `Dataset` PyTorch

In [ ]:
class SimpleDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y) if y is not None else None

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        return self.X[idx]

# DataLoaders
train_loader = DataLoader(SimpleDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
dev_loader = DataLoader(SimpleDataset(X_dev, y_dev), batch_size=BATCH_SIZE, shuffle=False)

print(f"Nombre de batchs par epoch : {len(train_loader)}")

## 4. Définition du modèle (avec `NUM_LAYERS` couches)

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size=4, hidden_size=50, num_layers=3, dropout_prob=0.2):
        super().__init__()
        layers = []
        in_features = input_size

        for _ in range(num_layers):
            layers.append(nn.Linear(in_features, hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_prob))
            in_features = hidden_size

        layers.append(nn.Linear(hidden_size, 3))  # 3 classes (Iris)
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# Instanciation
model = NeuralNet(
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    dropout_prob=DROPOUT_PROB
).to(DEVICE)

print(model)

## 5. Fonction de coût et optimiseur

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

## 6. Fonction d’entraînement (1 epoch)

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * X_batch.size(0)
    return total_loss / len(loader.dataset)

## 7. Fonction d’évaluation

In [ ]:
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == y_batch).sum().item()
    return correct / len(loader.dataset)

## 8. Entraînement complet + Visualisation

In [ ]:
train_losses = []
dev_accuracies = []

print("Début de l'entraînement...\n")
for epoch in range(1, NUM_EPOCHS + 1):
    loss = train_epoch(model, train_loader, criterion, optimizer, DEVICE)
    acc = evaluate(model, dev_loader, DEVICE)

    train_losses.append(loss)
    dev_accuracies.append(acc)

    if epoch % 10 == 0 or epoch <= 3:
        print(f"Epoch {epoch:2d} | Loss: {loss:.4f} | Accuracy: {acc:.4f}")

print(f"\nMeilleure accuracy : {max(dev_accuracies):.4f}")

## 9. Visualisation des courbes

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss', color='blue')
plt.title('Évolution de la perte')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(dev_accuracies, label='Dev Accuracy', color='green')
plt.title('Évolution de la précision')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

## Bilan 

Vous savez maintenant :
- Créer un `Dataset` et `DataLoader`
- Construire un modèle avec plusieurs couches
- Entraîner avec `Adam` et `CrossEntropyLoss`
- Évaluer et visualiser les performances

**À vous de jouer avec d’autres datasets !**